In [4]:
import time
import pygame,random,threading
from tkinter import Scale, Tk, messagebox,HORIZONTAL
import tkinter as tk
import sys
sys.setrecursionlimit(1000) #例如这里设置为25万 

class MazeGenerator:
    def __init__(self):
        pygame.init()
        self.clock = pygame.time.Clock()
        self.tile_size=23
        self.margin=1

        match goto:=2:
            case 1:#option one: design rows and cols yourself
                self.maze_rows,self.maze_cols=13,13# both rows and cols better be odd nums
                w,h= self.maze_cols*self.tile_size,self.maze_rows*self.tile_size
                self.window = pygame.display.set_mode((w,h))
                self.background= pygame.Surface(self.window.get_size())
                self.auto_maze_padding_y,self.auto_maze_padding_x=(h-self.maze_rows*self.tile_size)/2,(w-self.maze_cols*self.tile_size)/2
            case 2:#option two: design width and height yourself
                self.window = pygame.display.set_mode((827, 871))
                self.background= pygame.Surface(self.window.get_size())
                (w,h)= self.window.get_size()
                self.maze_rows,self.maze_cols=h // self.tile_size,w // self.tile_size
                self.auto_maze_padding_y,self.auto_maze_padding_x=(h-self.maze_rows*self.tile_size)/2,(w-self.maze_cols*self.tile_size)/2

        pygame.display.set_caption(f"width:{w},height:{h},rows:{self.maze_rows},cols:{self.maze_cols}")
        # pygame.display.set_allowed(FULLSCREEN | DOUBLEBUF | HWSURFACE | NOFRAME | TOPMOST)
        # No use anymore

        # 创建棋盘格状的背景
        # tiles = [((x * tile_size, y * tile_size, tile_size, tile_size), bgcolor1 if (x + y) % 2 == 0 else bgcolor2) for x in range((w + tile_size - 1) // tile_size) for y in range((h + tile_size - 1) // tile_size)]
        self.tiles=[]

        # Try to block the edges
        # mask=[[1 if x==0 or x==maze_rows-1 or y==0 or y==maze_cols-1 else 0  for y in range(maze_cols)] for x in range(maze_rows)]
        self.mask=[[1 for y in range(self.maze_cols)] for x in range(self.maze_rows)]

        self.toggle=1
        self.toggle_intensity=0
        self.ping=5
        self.generate_start=0
        self.path=[(1,1)]
        self.mask[1][1]=0

        self.click_one=(-1,-1)
        self.click_two=(-1,-1)
        self.solving=False

        self.start_color=(255,0,0)
        self.end_color=(0,255,0)
        self.result_maximun_length=-1
        self.draw_step=-1
        self.color_mask=[[-1 for y in range(self.maze_cols)] for x in range(self.maze_rows)]
        self.draw_control=-1

        self.quantum_tunneling=0.995

        self.toggle_node=[]

        self.generate_path()
        
        # pass
        # print("MazeGenerator Class instanced")

    def reset(self):
        self.click_one=(-1,-1)
        self.click_two=(-1,-1)
        self.solving=False
        self.result_maximun_length=-1
        self.draw_step=-1
        self.color_mask=[[-1 for y in range(self.maze_cols)] for x in range(self.maze_rows)]
        self.draw_control=-1
        print('reset called')

    def blend_color(self,ratio):
        R1,G1,B1=self.start_color
        R2,G2,B2=self.end_color
        return (int(R1*ratio+R2*(1-ratio)),int(G1*ratio+G2*(1-ratio)),int(B1*ratio+B2*(1-ratio)))

    def light_dark_color(self,color,ratio):
        return (int(color[0]*ratio),int(color[1]*ratio),int(color[2]*ratio))

    def mask2tiles(self):
        
        if self.draw_control!=-1:
            self.draw_control+=1
            if self.draw_control%self.ping==0:
                self.draw_control=0
                self.result_animation()

        self.tiles=[]
        for x in range(self.maze_rows):
            for y in range(self.maze_cols):
                if self.color_mask[x][y]!=-1:
                    fill_color=self.blend_color(self.color_mask[x][y]/(self.result_length_max-1))
                else:
                    match self.mask[x][y]:
                        case 0: fill_color=(0, 0, 0)
                        case 1: fill_color=(192, 192, 192)
                if (x,y) in self.toggle_node or self.click_one!=(-1,-1) and (x,y)==self.click_one or self.click_two!=(-1,-1) and (x,y)==self.click_two:
                    if self.toggle_intensity+self.toggle/self.ping>1:
                        self.toggle=-1
                    elif self.toggle_intensity+self.toggle/self.ping<0:
                        self.toggle=1
                    self.toggle_intensity=self.toggle_intensity+self.toggle/self.ping
                    fill_color=self.light_dark_color(fill_color,self.toggle_intensity)
                self.tiles.append((( y * self.tile_size+self.margin+self.auto_maze_padding_y,x * self.tile_size+self.margin+self.auto_maze_padding_x, self.tile_size-self.margin, self.tile_size-self.margin),fill_color))
        # 为棋盘添加小格子
        for rect, color in self.tiles:
            pygame.draw.rect(self.background, color, rect)
        self.window.blit(self.background, (0, 0))

    def return_xy(self,pos):
        '''Returns the index of row and col'''
        return int((pos[1]-self.auto_maze_padding_y)//self.tile_size),int((pos[0]-self.auto_maze_padding_x)//self.tile_size)

    def generate_path(self):
        
        # self.resursion_depth=0
        def call_back():
            # self.resursion_depth+=1
            # pygame.display.set_caption(f'recursion_depth: {self.resursion_depth}')
            # print(f'recursion_depth: {self.resursion_depth}')
            # time.sleep(3)
            available_direction=[]
            node_row,node_col=self.path[-1]
            if node_row-2>0:
                if self.mask[node_row-2][node_col]!=0  or random.random()>self.quantum_tunneling:
                    available_direction.append(0)
            if node_col-2>0:
                if self.mask[node_row][node_col-2]!=0 or random.random()>self.quantum_tunneling:
                    available_direction.append(1)
            if node_row+3<self.maze_rows:
                if self.mask[node_row+2][node_col]!=0 or random.random()>self.quantum_tunneling:
                    available_direction.append(2)
            if node_col+3<self.maze_cols:
                if self.mask[node_row][node_col+2]!=0 or random.random()>self.quantum_tunneling:
                    available_direction.append(3)

            if len(available_direction)==0:
                return

            direction=random.sample(available_direction,k=1)[0]
            match direction:
                case 0:next_coord=((node_row-2,node_col),(node_row-1,node_col))
                case 1:next_coord=((node_row,node_col-2),(node_row,node_col-1))
                case 2:next_coord=((node_row+2,node_col),(node_row+1,node_col))
                case 3:next_coord=((node_row,node_col+2),(node_row,node_col+1))
            self.path.append(next_coord[0])
            for every_coord in next_coord:
                self.mask[every_coord[0]][every_coord[1]]=0
            call_back()
        call_back()
        if len(self.path)>1: 
            self.path.pop() 
            self.generate_path()
        else: 
            self.generate_start=-1
            # self.show_message("Call-back reaches end, Maze has been generated !")

    def result_animation(self):
        if self.draw_step+1==self.result_length_max:
            self.draw_control=-1
            self.solving=False
            return
        self.draw_step+=1
        self.toggle_node=[]
        for result in self.results: 
            if self.draw_step<len(result):
                # print('result_animation called',self.draw_step,len(result),self.result_length_max,self.blend_color(self.draw_step/(self.result_length_max-1))) 
                draw_node=result[self.draw_step]
                # if self.color_mask[draw_node[0]][draw_node[1]]!=-1 and self.color_mask[draw_node[0]][draw_node[1]]<self.draw_step:
                if self.color_mask[draw_node[0]][draw_node[1]]==-1 or  self.draw_step < self.color_mask[draw_node[0]][draw_node[1]]:
                    self.color_mask[draw_node[0]][draw_node[1]]=self.draw_step  #at least I know this thing starts with zero
                    self.toggle_node.append((draw_node[0],draw_node[1]))

    def solver(self):
    
        start_pos=self.click_one
        end_pos=self.click_two
        # self.click_one,self.click_two=(-1,-1),(-1,-1)
        self.solving=True
        pygame.display.set_caption(f'Solving, this is scandalous.')

        self.result=[]
        self.resursion_depth=0

        def call_back(start_pos,trace):
            self.resursion_depth+=1
            if self.resursion_depth%10000==0:
                pygame.display.set_caption(f'recursion_depth reached 10000, input anything to continue')
                input()
            pygame.display.set_caption(f'recursion_depth: {self.resursion_depth}')
            if start_pos==end_pos:
                self.result.append(trace)
                return
            available_direction=[]
            node_row,node_col=start_pos
            if node_row-1>0 and self.mask[node_row-1][node_col]==0:
                available_direction.append(0)
            if node_col-1>0 and self.mask[node_row][node_col-1]==0:
                available_direction.append(1)
            if node_row+2<self.maze_rows and self.mask[node_row+1][node_col]==0:
                available_direction.append(2)
            if node_col+2<self.maze_cols and self.mask[node_row][node_col+1]==0:
                available_direction.append(3)

            if len(available_direction)==0:
                return

            def puppet(direction):
                match direction:
                    case 0:next_coord=(node_row-1,node_col)
                    case 1:next_coord=(node_row,node_col-1)
                    case 2:next_coord=(node_row+1,node_col)
                    case 3:next_coord=(node_row,node_col+1)
                return next_coord

            for direction in available_direction:
                nextNode=puppet(direction)
                if nextNode not in trace:
                    call_back(nextNode,trace+[nextNode,])
            
        call_back(start_pos,[start_pos,])
        
        # print(self.result)
        # print()
        # print(len(self.result))
        # self.result.sort()

        self.result_length_map=[len(x) for x in self.result]
        print(self.result_length_map)
        if len(self.result_length_map)==1:
            self.results=self.result
        else:
            self.results=[min(self.result),max(self.result)]
            
        print(self.results)
        self.result_length_min=min(self.result_length_map)
        self.result_length_max=max(self.result_length_map)

        self.draw_control=1
        pygame.display.set_caption(f'Found {len(self.result_length_map)} answer(s), Easy peasy, lemon squeezy!.')

    def start(self):
        run = True
        while run:
            self.clock.tick(60)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False
                elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                        ## if mouse is pressed get position of cursor ##
                        pos = pygame.mouse.get_pos()
                        if not self.solving:
                            coord=self.return_xy(pos)
                            if self.click_one!=(-1,-1) and self.click_two!=(-1,-1):
                                pygame.display.set_caption(f'Cleared all marks, choose again')
                                self.reset()
                            if self.mask[coord[0]][coord[1]]==0:
                                if self.click_one==(-1,-1):
                                    self.click_one=coord
                                    pygame.display.set_caption(f'One Clicked, Mask coord as {coord}')
                                elif coord==self.click_one:
                                    pygame.display.set_caption(f'Click another spot, this is not how it works')
                                elif self.click_two==(-1,-1):
                                    self.click_two=coord
                                    self.solver()
                                ## check if cursor is on tiles ##

            self.mask2tiles()
            # 将棋盘背景绘制到窗口上
            pygame.display.flip()
        pygame.quit()

    def exit(self):
        pygame.quit()

    def __del__(self):
        pass
        # print("MazeGenerator Class instance destroyed")
        
maze=MazeGenerator()
maze.start()

[239, 235]
[[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 5), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 9), (5, 9), (5, 10), (5, 11), (6, 11), (7, 11), (7, 10), (7, 9), (7, 8), (7, 7), (8, 7), (9, 7), (10, 7), (11, 7), (11, 6), (11, 5), (11, 4), (11, 3), (10, 3), (9, 3), (9, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (19, 2), (19, 3), (20, 3), (21, 3), (21, 4), (21, 5), (21, 6), (21, 7), (22, 7), (23, 7), (23, 8), (23, 9), (23, 10), (23, 11), (24, 11), (25, 11), (25, 12), (25, 13), (24, 13), (23, 13), (23, 14), (23, 15), (24, 15), (25, 15), (25, 16), (25, 17), (26, 17), (27, 17), (27, 18), (27, 19), (26, 19), (25, 19), (25, 20), (25, 21), (25, 22), (25, 23), (24, 23), (23, 23), (23, 22), (23, 21), (22, 21), (21, 21), (20, 21), (19, 21), (19, 20), (19, 19), (19, 18), (19, 17), (20, 17), (21, 17), (21, 16), (21, 15), (20, 15), (19, 15), (19, 14), (19, 13), (19, 12), (19, 11), (19, 10), (19, 9), (18, 9), (17, 9), (16, 9), (15, 9)

In [33]:
maze.exit()

In [2]:
maze.toggle_node

[]

In [6]:
your_list=[['Hello', 'world'], ['how','are','you'], ['have','a','good','day']]
max(your_list, key=len)
len(max(your_list, key=len))

4